In [60]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [126]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Dropout
from tensorflow.keras.activations import relu
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

In [227]:
data=pd.read_csv("Filtered_Interpolation.csv")

In [228]:
# data.drop("Unnamed: 0",inplace=True,axis=1)
data

,Time,Bx,By,Bz,1,2,3,4,5,6,...,46,47,48,49,50,year,month,day,days,Kp
0,2016-01-01 03:04:00,3.354850,0.889208,-12.89340,69.233568,28.595303,79.281279,67.567974,80.387082,68.544143,...,317.913009,310.680832,285.258484,327.421508,287.222025,2016.0,1.0,1.0,30681.125,5.333
1,2016-01-01 03:05:00,3.230850,0.825016,-12.88540,67.712974,26.073684,78.824265,67.994133,81.572727,69.103572,...,317.614327,312.594745,288.726355,328.825933,287.983383,NaN,NaN,NaN,NaN,NaN
2,2016-01-01 03:06:00,3.336180,0.858471,-12.85350,67.434152,27.499140,79.388494,65.546215,82.184906,66.572431,...,317.769843,312.998135,287.243899,327.301491,287.368032,NaN,NaN,NaN,NaN,NaN
3,2016-01-01 03:07:00,3.288260,0.785345,-12.87140,68.448095,28.245070,80.528583,65.765642,82.328225,67.729068,...,317.724181,313.215421,287.386911,328.485867,285.967557,NaN,NaN,NaN,NaN,NaN
4,2016-01-01 03:08:00,3.281750,0.790407,-12.85160,65.810425,27.875827,79.179637,66.631353,81.177715,70.074515,...,319.337578,312.964716,287.347357,327.080587,288.261405,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148324,2023-04-28 09:05:00,1.030660,-1.886720,-2.15262,2.550380,0.231726,3.782020,2.191550,8.743940,0.417479,...,318.900342,310.869439,287.816010,327.774118,287.812586,NaN,NaN,NaN,NaN,NaN
148325,2023-04-28 09:06:00,-0.065671,-0.462128,3.22039,2.175760,0.231726,10.140600,4.089990,3.885790,0.348397,...,317.868607,314.254726,287.109078,326.338215,286.678696,NaN,NaN,NaN,NaN,NaN
148326,2023-04-28 09:07:00,0.494277,-1.058260,2.71487,2.056990,0.231726,5.209680,1.372850,8.423130,0.349598,...,316.635300,311.443918,286.197754,328.620907,288.567408,NaN,NaN,NaN,NaN,NaN
148327,2023-04-28 09:08:00,1.287520,-2.142040,2.22568,2.008430,0.231726,5.980320,4.334420,9.921030,0.345248,...,317.259322,312.227606,287.794036,328.865222,286.640490,NaN,NaN,NaN,NaN,NaN


In [165]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150060 entries, 0 to 150059
Data columns (total 59 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    150060 non-null  object 
 1   Bx      150060 non-null  float64
 2   By      150060 non-null  float64
 3   Bz      150060 non-null  float64
 4   1       150060 non-null  float64
 5   2       150060 non-null  float64
 6   3       150060 non-null  float64
 7   4       150060 non-null  float64
 8   5       150060 non-null  float64
 9   6       150060 non-null  float64
 10  7       150060 non-null  float64
 11  8       150060 non-null  float64
 12  9       150060 non-null  float64
 13  10      150060 non-null  float64
 14  11      150060 non-null  float64
 15  12      150060 non-null  float64
 16  13      150060 non-null  float64
 17  14      150060 non-null  float64
 18  15      150060 non-null  float64
 19  16      150060 non-null  float64
 20  17      150060 non-null  float64
 21  18      15

In [229]:
k=np.random.randint(2,155,24)
k

array([148,  79,  22,  38,  83,  76,   2,  79,  57, 152,  20, 109,  77,
        53,  73,  95,  94,  76,  52, 145,  36,   7, 145,  52])

In [239]:

def train_gen(dftrain,X_train,y_train):
    a=len(lol)
    index = lol.index
    print(a)
    for i in range(0,a):
        # print(i)
        try: 
            # k=183
            print(i)
            c=index[i]
            # print("hi ",c)
            l=index[i+1]
            df=dftrain.iloc[c:l]
            # print(len(df),"Length of df ")
            # print("Going",len(df))
            if(len(df)<183):
                k=np.random.randint(0,159,24)
                # print(len(df.iloc[k]),"dflength to be added ")
                df = pd.concat([df,df.iloc[k]],axis=0)
            print(len(df),"length after concatenating")
            X_train.append(df.drop(['year', 'month', 'day', 'days', 'Kp','Time'],axis=1))
        except Exception as e:
            print(e)
        print("-------------------------------------------------")

In [231]:
y=data["Kp"]

In [232]:
lol = y[~(pd.isna(y))]

In [233]:
lol.index

Int64Index([     0,    183,    366,    549,    732,    915,   1074,   1257,
              1440,   1623,
            ...
            146571, 146730, 146913, 147096, 147255, 147438, 147621, 147804,
            147963, 148146],
           dtype='int64', length=823)

In [234]:
data.iloc[[0,2]]

,Time,Bx,By,Bz,1,2,3,4,5,6,...,46,47,48,49,50,year,month,day,days,Kp
0,2016-01-01 03:04:00,3.35485,0.889208,-12.8934,69.233568,28.595303,79.281279,67.567974,80.387082,68.544143,...,317.913009,310.680832,285.258484,327.421508,287.222025,2016.0,1.0,1.0,30681.125,5.333
2,2016-01-01 03:06:00,3.33618,0.858471,-12.8535,67.434152,27.499140,79.388494,65.546215,82.184906,66.572431,...,317.769843,312.998135,287.243899,327.301491,287.368032,NaN,NaN,NaN,NaN,NaN


In [240]:
X_train=[]
y_train=[]
train_gen(data,X_train,y_train)

823
0
183 length after concatenating
-------------------------------------------------
1
183 length after concatenating
-------------------------------------------------
2
183 length after concatenating
-------------------------------------------------
3
183 length after concatenating
-------------------------------------------------
4
183 length after concatenating
-------------------------------------------------
5
183 length after concatenating
-------------------------------------------------
6
183 length after concatenating
-------------------------------------------------
7
183 length after concatenating
-------------------------------------------------
8
183 length after concatenating
-------------------------------------------------
9
183 length after concatenating
-------------------------------------------------
10
183 length after concatenating
-------------------------------------------------
11
183 length after concatenating
------------------------------------------------

In [238]:
# X_train= np.array(X_train)
len(X_train)
# len(y_train)

822

In [242]:
lol=lol[:-1]
lol

0         5.333
183       5.000
366       1.667
549       1.000
732       2.000
          ...  
147255    8.333
147438    6.667
147621    3.333
147804    3.000
147963    4.000
Name: Kp, Length: 822, dtype: float64

In [243]:
Y=lol
Y

0         5.333
183       5.000
366       1.667
549       1.000
732       2.000
          ...  
147255    8.333
147438    6.667
147621    3.333
147804    3.000
147963    4.000
Name: Kp, Length: 822, dtype: float64

In [47]:
pd.DataFrame.to_numpy(X_train[0])
arr = X_train[0].to_numpy()

array([[  3.35485   ,   0.889208  , -12.8934    , ..., 288.52229341,
        327.79052681, 286.27082485],
       [  3.23085   ,   0.825016  , -12.8854    , ..., 286.66801451,
        329.75626119, 287.30391508],
       [  3.33618   ,   0.858471  , -12.8535    , ..., 286.07256126,
        327.88502165, 288.72463901],
       ...,
       [  4.58858   ,  -7.81308   ,  -5.78437   , ..., 136.654     ,
        155.294     , 168.134     ],
       [  4.29098   ,  -6.4471    ,  -7.19206   , ..., 135.228     ,
        161.165     , 140.635     ],
       [  3.2772    ,  -2.49201   ,  -9.90422   , ..., 139.984     ,
        157.239     , 141.956     ]])

In [244]:
Scaler=MinMaxScaler()

In [247]:
X_train

[          Bx        By        Bz           1           2           3  \
 0    3.35485  0.889208 -12.89340   69.233568   28.595303   79.281279   
 1    3.23085  0.825016 -12.88540   67.712974   26.073684   78.824265   
 2    3.33618  0.858471 -12.85350   67.434152   27.499140   79.388494   
 3    3.28826  0.785345 -12.87140   68.448095   28.245070   80.528583   
 4    3.28175  0.790407 -12.85160   65.810425   27.875827   79.179637   
 ..       ...       ...       ...         ...         ...         ...   
 178  4.32586 -8.008960  -5.69739  137.917000   99.008300  132.370000   
 179  4.37219 -7.858180  -5.82984  144.509000  104.199000  125.531000   
 180  4.58858 -7.813080  -5.78437  143.500000  102.396000  128.116000   
 181  4.29098 -6.447100  -7.19206  143.834000  109.560000  125.475000   
 182  3.27720 -2.492010  -9.90422  148.235000  104.279000  121.785000   
 
               4           5           6           7  ...          41  \
 0     67.567974   80.387082   68.544143   86.341

In [248]:
#Train_data=pd.DataFrame.to_numpy(X_train[0])
Train_data=[]
for i in range(1, len(Y)):
    #Train_data = np.concatenate([Train_data, pd.DataFrame.to_numpy(X_train[i])], axis=0)
    x1=Scaler.fit_transform(pd.DataFrame.to_numpy(X_train[i]))
    Train_data.append(x1)
Train_data=np.array(Train_data)

x_train= Train_data[0:780]
x_val= Train_data[781:]
y_tr=np.asarray(Y[:780])
y_val=np.asarray(Y[781:])

# y_tr_final = np.asarray(y_tr)

In [252]:
type(x_train[0][0][0])

numpy.float64

In [253]:
xcopy= x_train

In [144]:
# Scaler=MinMaxScaler()
# x_train=Scaler.fit_transform(x_train)

In [ ]:
type(y_tr_final)

numpy.ndarray

In [257]:
type(y_tr[0])

numpy.float64

In [168]:
x_train.shape

(780, 183, 53)

In [263]:
model=Sequential()
model.add(InputLayer((183,53)))
model.add(LSTM(100,input_shape=(183,53),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(50,activation=relu))
model.add(Dense(1,activation=relu))
model.summary()

# model = Sequential()
# model.add(InputLayer((183,53)))
# model.add(LSTM(units=50,input_shape=(183,53)))
# # model.add(LSTM(units
# model.add(Dense(32,'relu'))
# model.add(Dense(2,'linear'))

# model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_31 (LSTM)              (None, 183, 100)          61600     
                                                                 
 dropout_10 (Dropout)        (None, 183, 100)          0         
                                                                 
 lstm_32 (LSTM)              (None, 183, 100)          80400     
                                                                 
 lstm_33 (LSTM)              (None, 100)               80400     
                                                                 
 dropout_11 (Dropout)        (None, 100)               0         
                                                                 
 dense_22 (Dense)            (None, 50)                5050      
                                                                 
 dense_23 (Dense)            (None, 1)               

In [264]:
# model.compile(loss='mean_squared_error',optimizer="adam")
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.01),metrics =[RootMeanSquaredError()])

In [265]:
model.fit(x_train,y_tr,epochs=20,verbose=1,batch_size=1)


Epoch 1/20


780/780 [==============================] - 29s 34ms/step - loss: 2.4303 - root_mean_squared_error: 1.5590
Epoch 2/20
780/780 [==============================] - 26s 34ms/step - loss: 1.9909 - root_mean_squared_error: 1.4110
Epoch 3/20
780/780 [==============================] - 26s 34ms/step - loss: 1.9858 - root_mean_squared_error: 1.4092
Epoch 4/20
780/780 [==============================] - 26s 34ms/step - loss: 1.9320 - root_mean_squared_error: 1.3900
Epoch 5/20
780/780 [==============================] - 26s 34ms/step - loss: 1.9087 - root_mean_squared_error: 1.3816
Epoch 6/20
780/780 [==============================] - 26s 34ms/step - loss: 1.8286 - root_mean_squared_error: 1.3523
Epoch 7/20
780/780 [==============================] - 26s 34ms/step - loss: 1.8711 - root_mean_squared_error: 1.3679
Epoch 8/20
780/780 [==============================] - 26s 34ms/step - loss: 1.8412 - root_mean_squared_error: 1.3569
Epoch 9/20
780/780 [==============================] - 26s 34ms/step - loss:

In [266]:
pred = model.predict(x_val)

2/2 [==============================] - 1s 17ms/step


In [272]:
np.square(np.subtract(pred,y_val)).mean() 

4.0474009843654155